# Troubleshooting Authentication with DefaultAzureCredential

## Overview

DefaultAzureCredential is the recommended way to handle authentication in Azure applications. It provides a streamlined authentication flow by trying multiple credential types in sequence until one succeeds. This notebook will help you troubleshoot common authentication issues and ensure proper setup.

## Understanding DefaultAzureCredential

DefaultAzureCredential attempts authentication methods in the following order:

1. Environment Credentials
2. Workload Identity (in Kubernetes)
3. Managed Identity
4. Azure CLI Credentials
5. Azure PowerShell Credentials
6. Visual Studio Code Credentials
7. Interactive Browser Authentication (as fallback)

## Prerequisites

Ensure you have the following installed:
- Azure CLI
- Azure Developer CLI (optional)
- Python Virtual Environment or Conda (use `uv venv` or `conda create`)
- Required role assignments (Azure AI Developer)
- Jupyter Notebook environment - kernel configured to use Python 3.8 or later


# First, we'll authenticate using Azure CLI
This is the recommended approach for local development.

When you run the code below, you will be redirected to:
- Either the Azure portal in your browser to complete the login 
- Or use Windows login if you're already signed in to your machine

The code will:
1. Load environment variables from .env file, including the TENANT_ID
2. Use Azure CLI to log in to your specific tenant  
3. Test authentication by attempting to get a token


In [ ]:
# Import required packages
from IPython.display import display
from IPython.display import HTML
import getpass
from dotenv import load_dotenv
import os
from pathlib import Path  # For cross-platform path handling

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv(parent_dir / '.env')  # Load environment variables from .env file

# Get tenant ID from environment variable
tenant_id = os.getenv("TENANT_ID")

# Azure login with specific tenant
!az login --tenant {tenant_id}

# Get subscription ID from connection string
conn_str = os.getenv("PROJECT_CONNECTION_STRING")
subscription_id = conn_str.split(';')[1] if conn_str else None

if subscription_id:
    # Set the subscription
    !az account set --subscription {subscription_id}
    print(f"✓ Successfully set subscription: {subscription_id}")
else:
    print("⚠️ Could not get subscription ID from PROJECT_CONNECTION_STRING")


# Next, we'll test the authentication by attempting to get a token using AzureCliCredential

AzureCliCredential will try multiple authentication methods in this order:
1. Environment credentials (if environment variables are set)
2. Managed Identity credentials (if running in Azure)
3. Shared Token Cache credentials (from previous logins) 
4. Visual Studio Code credentials (if using VS Code)
5. Azure CLI credentials (which we just set up)

The code below will:
1. Create a AzureCliCredential instance
2. Try to get a token for Azure Cognitive Services
3. Print success message if token is acquired

>Note: You may see some warning/error messages as it tries different authentication methods - 
>this is normal and can be ignored as long as you see "Successfully acquired token!" at the end


In [ ]:
# Then use AzureCliCredential in your code
from azure.identity import AzureCliCredential
from azure.core.credentials import AccessToken
import logging

# Enable detailed logging
logging.basicConfig(level=logging.DEBUG)

try:
    credential = AzureCliCredential()
    # Test token acquisition
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    print("Successfully acquired token!")
except Exception as e:
    print(f"Authentication failed: {str(e)}")

## Now that you have successfully authenticated, you can proceed to [2-environment_setup.ipynb](2-environment_setup.ipynb), or try the additional authentication methods or troubleshoot below.


## Common Issues and Solutions

1. **Token Acquisition Failed**
   - Verify Azure CLI login: `az login --tenant <tenant-id>`
   - Check default subscription: `az account show`
   - Ensure correct tenant: `az account set --subscription <subscription-id>`

2. **Missing Role Assignments**
   - Add Azure AI Developer role:
   ```bash
   az role assignment create --assignee "user@domain.com" \
       --role "Azure AI Developer" \
       --scope "/subscriptions/<subscription-id>/resourceGroups/<resource-group>/providers/Microsoft.CognitiveServices/accounts/<resource-name>"
   ```

3. **Environment Variable Issues**
   - Verify environment variables are set correctly
   - Check for typos in variable names
   - Ensure no extra spaces in values

## Best Practices

1. Always use environment variables for service principal credentials
2. Implement proper error handling and logging
3. Use managed identities when deploying to Azure services
4. Regularly rotate service principal secrets
5. Follow the principle of least privilege when assigning roles

## Additional Resources

- [Azure Identity Documentation](https://docs.microsoft.com/python/api/overview/azure/identity-readme)
- [DefaultAzureCredential Authentication Flow](https://docs.microsoft.com/azure/developer/python/azure-sdk-authenticate)
- [Azure RBAC Documentation](https://docs.microsoft.com/azure/role-based-access-control/overview)